In [17]:
import os

import tensorflow as tf

In [2]:
tfrecords_filename = 'bag_of_popcorns_indexing.tfrecords'

In [3]:
dataset = tf.data.TFRecordDataset([tfrecords_filename])

In [4]:
def parser(serialized_example):
    """Parses a single tf.Example into image and label tensors."""
    features = tf.parse_single_example(
      serialized_example,
      features={
        'review': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64),
        'length': tf.FixedLenFeature([], tf.int64)
        })
    
    review = tf.decode_raw(features['review'], tf.int32)
    review = tf.cast(review, tf.int64)
    label = tf.cast(features['label'], tf.int32)
    length = tf.cast(features['length'], tf.int32)
  
    return {'x': review, 'len': length}, label

dataset = dataset.map(parser)

In [12]:
iterator = dataset.make_one_shot_iterator()

In [13]:
d = iterator.get_next()

sess = tf.Session()

print(sess.run(d))

({'x': array([[  497,     2,   109, ...,     0,     0,     0],
       [  322,   205,     2, ...,     0,     0,     0],
       [  341,    60, 10610, ...,     0,     0,     0],
       ...,
       [ 5640,  9348, 11094, ...,   174, 21811,   176],
       [16963,   102,   502, ...,     0,     0,     0],
       [   18,  4828,  3633, ...,     0,     0,     0]]), 'len': array([ 50, 125, 152, 139,  96, 200,  79,  38,  71,  47, 200, 108,  86,
       117,  96,  85,  49,  85,  29,  79, 178,  64,  32, 105, 139, 134,
        19,  35,  29, 160, 200,  88,  30,  50,  95, 137, 200,  74, 140,
        62,   8,  79, 102, 101, 200, 149, 200, 123, 153,  76,  33,  50,
        88, 200, 115, 200,  91,  35,  48,  72, 131,  63,  59,  52,  72,
       200,  84,  87,  61, 200,  97,  35,  29,  30,  99,  93,  69, 197,
        35, 169,  66,  85,  63, 156, 143, 200,  44,  34, 159, 116,  90,
       163, 104, 114,  66, 200,  73, 200, 127, 102], dtype=int32)}, array([1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 

In [14]:
def train_input_fn():
    tfrecords_filename = 'bag_of_popcorns_indexing.tfrecords'
    dataset = tf.data.TFRecordDataset([tfrecords_filename])
    dataset = dataset.map(parser)
    dataset = dataset.shuffle(buffer_size=100)
    dataset = dataset.batch(100)
    dataset = dataset.repeat()
   
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [29]:
# 여러 모델을 저장할 것을 대비해 dictionary 변수를 선언해 둡니다.
all_classifiers = {}

# 이 함수는 학습과 validation 성능을 확인한 함수입니다.
# input 파라메터로 estimator 객체를 받아 학습을 합니다.
def train_and_evaluate(classifier):
    # 예측 테스트를 위해 모델을 학습시키고 저장합니다.
    all_classifiers[classifier.model_dir] = classifier
    # estimator 객체를 통해 학습을 합니다. input function을 지정해주고 1 step에 대한 학습을 시도합니다.
    classifier.train(input_fn=train_input_fn, steps=1000)
    # evluation을 합니다.
    #eval_results = classifier.evaluate(input_fn=eval_input_fn)
    #predictions = np.array([p['logistic'][0] for p in classifier.predict(input_fn=eval_input_fn)])
    
    # name scopes의 재사용을 위해 graph를 reset합니다.
    tf.reset_default_graph()
    
    # 학습한 모델 graph를 저장합니다.
    #with tf.Session() as sess:
    #    writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
    #    writer.add_summary(sess.run(pr), global_step=0)
    #    writer.close()

In [21]:
model_dir = './checkpoint/rnn_classifier'
vocab_size = 150000
sentence_size = 200
embedding_size = 50

# pre-made estimator로 평가를 할 때, 일정한 함수를 사용하게 세팅합니다.
head = tf.contrib.estimator.binary_classification_head()

def rnn_model_fn(features, labels, mode, params):
    #embedding layer를 선언합니다.
    input_layer = tf.contrib.layers.embed_sequence(
                    features['x'],
                    vocab_size,
                    embedding_size,
                    initializer=params['embedding_initializer']
                    )
    # 현재 모델이 학습모드인지 여부를 확인하는 변수입니다.
    training = (mode == tf.estimator.ModeKeys.TRAIN)
    # embedding layer에 대한 output에 대해 dropout을 취합니다.
    dropout_emb = tf.layers.dropout(inputs=input_layer,
                                   rate=0.2,
                                   training=training)
    
    # LSTM Cell Network를 선언합니다. 객체 생성자 파라메터에는 해당 hidden dimension을 정의해둡니다.
    # Multi Layer의 경우엔 아래와 같이 List 형태로 저장을 해둡니다.
    rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [100, 100]]

    # Multi RNN Cell이 복수의 LSTM Cell를 하나의 Cell Network로 선언해줍니다.
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

    # 네트워크와 embedding vector와 연산하기 위해 dynamic_rnn 함수를 선언합니다.
    outputs, state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                       inputs=dropout_emb,
                                       dtype=tf.float32)
    
    # rnn network를 거쳐 나온 마지막 step의 hidden vector를 활용하여 감정을 분석하고자 합니다.
    # rnn network에서 나온 벡터들 중 마지막 step에 있는 vector만을 가지고 fully connected를 거치도록 합니다.
    hidden = tf.layers.dense(inputs=outputs[:,-1,:], units=100, activation=tf.nn.relu)  
    dropout_hidden = tf.layers.dropout(inputs=hidden, rate=0.2, training=training)
    # 최종적으로 1 dim의 output만 할 수 있도록 dense layer를 활용하여 차원 변환을 해줍니다.
    logits = tf.layers.dense(inputs=dropout_hidden, units=1)
    
    # evaluation을 하는 상황이 아닌 경우 label 벡터에 대한 값의 dimension을 맞춰줍니다.
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
    
    # Optimizer를 지정해줍니다. 하이퍼파라메터로 다른 Optimizer를 지정해줄 수 있습니다.
    optimizer = tf.train.AdamOptimizer() 
    
    def _train_op_fn(loss):
        return optimizer.minimize(
                loss=loss,
                global_step=tf.train.get_global_step())

    # pre-made estimator를 활용하여 train 및 evaluation을 진행합니다.
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits,
        train_op_fn=_train_op_fn)

# estimator에는 params라는 파라메터를 전달할 수 있도록 정의되어있습니다. 
# 여러분이 해당 모델에 설정하고자 하는 하이퍼파라메터에 대해 각각 지정할 내용을 dictionary 형태로 
# 선언을 하면 모델 함수 내부에서 활용이 가능합니다.

# 여기선 embedding layer에 파라메터에 대해 initialize를 설정해줍니다.
# 이 경우는 -1~1 사이의 uniform 분포로 초기화 합니다.
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}

# tf.estimator 객체를 선언합니다. 모델과 저장할 모델 저장 위치를 선언해줍니다.
rnn_classifier = tf.estimator.Estimator(model_fn=rnn_model_fn,
                                        model_dir=os.path.join(model_dir, 'rnn'),
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './checkpoint/rnn_classifier/rnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11f538f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [30]:
# 학습을 진행합니다.
train_and_evaluate(rnn_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoint/rnn_classifier/rnn/model.ckpt-103
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 104 into ./checkpoint/rnn_classifier/rnn/model.ckpt.
INFO:tensorflow:loss = 68.56486, step = 104
INFO:tensorflow:global_step/sec: 2.60613
INFO:tensorflow:loss = 68.25571, step = 204 (38.373 sec)
INFO:tensorflow:global_step/sec: 2.54038
INFO:tensorflow:loss = 69.41705, step = 304 (39.363 sec)
INFO:tensorflow:global_step/sec: 2.7804
INFO:tensorflow:loss = 70.32014, step = 404 (35.966 sec)
INFO:tensorflow:global_step/sec: 2.61448
INFO:tensorflow:loss = 68.89592, step = 504 (38.249 sec)
INFO:tensorflow:global_step/sec: 2.7302
INFO:tensorflow:loss = 70.51912, step = 604 (36.628 sec)
INFO:tensorflow:global_step/sec: 2.49799
INFO:tens

KeyboardInterrupt: 